# VDS 1022/i Oscilloscope

See `vds1022.html` for the documentation.

The following modules are used in this Jupyter notebook :
 * `pyusb` `numpy` `pandas` `streamz` `bokeh` `matplotlib` `hvplot`

Note that the first call of `VDS1022()` will take a few seconds since it has to load the FPGA firmware.
The device will remain connected unless `.dispose()` is called or if the kernel is restarted. Interrupting the kernel (`I`,`I`) will not terminate the connection.

### Examples :
1. [Live plotting](#1.-Live-plotting)
2. [Acquisition](#2.-Acquisition)
3. [Data logging](#3.-Data-logging)
4. [Continuous sampling](#4.-Continuous-sampling)


## 1. Live plotting


Plot the signals and refresh every 200ms

In [14]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('20ms')
dev.set_channel(CH1, range='10v', coupling=DC, offset=0, probe='x10')
dev.set_channel(CH2, range='5v', coupling=DC, offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='2.5v', sweep=AUTO)

dev.plot();

In [15]:
dev.stop();

## 2. Acquisition
Pull the buffer of the device once triggered and plot.

In [8]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('5ms')
dev.set_channel(CH1, range='10v', coupling=DC, offset=0, probe='x10')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, position=0.5, level='2.5v', sweep=ONCE)
frames = dev.pull()
frames.plot()

Generate descriptive statistics.

In [9]:
frames.describe()

,CH1,CH2
Count,5000.000000,5000.000000
Vavg,0.007000,-0.454000
Vrms,14.083000,19.533000
Vamp,39.200000,55.200000
Vbase,-19.600000,-27.600000
Vtop,19.600000,27.600000
Vpp,39.200001,55.200001
Vmin,-19.600000,-27.600000
Vmax,19.600000,27.600000
Period,0.001667,0.001667


Custom measures.

In [10]:
(
    'CH1 Vmax: %sV'  % frames.CH1.max(),
    'CH1 Vavg: %sV' % frames.CH1.mean(),
    'CH1 vRms: %sV'  % frames.CH1.rms()
)

('CH1 Vmax: 19.6V', 'CH1 Vavg: 0.007V', 'CH1 vRms: 14.083V')

Convert to [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe)

In [11]:
frames.to_dataframe()

,CH1,CH2
-0.002500,-4.0,20.0
-0.002499,-4.0,19.6
-0.002498,-4.0,19.6
-0.002497,-4.0,19.6
-0.002496,-4.0,19.6
...,...,...
0.002496,3.6,25.6
0.002497,3.2,25.6
0.002498,3.2,25.6
0.002499,3.2,25.6


## 3. Data logging
This section provides examples to measure a signal at a defined interval.

Measure and plot the RMS voltage.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('5ms')
dev.set_channel(CH1, range='10v', coupling=DC, offset=0, probe='x1')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x1')
dev.stream(freq=1).agg('rms').plot();

Measure and print the RMS voltage.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('5ms')
dev.set_channel(CH1, range='10v', coupling=DC, offset=0, probe='x1')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x1')

def handler(frames) :
    args = round(frames.time, 1), frames.CH1.rms()
    print('%s  CH1:%sv     ' % args, end='\r')

dev.stream(freq=1).sink(handler);

## 4. Continuous sampling

This section provides examples to aquire and plot continuous samples without interruption for a defined duration or on a trigger event.  
While this device can acquire sampling frames at 100Ms/s, the maximum continuous sampling rate is around 100Ks/s (20ms timerange).  
Since the amount of data can be quite consequent, the plotting has to go through rasterization.

Read continuous samples for a defined time

In [ ]:
from vds1022 import *
import hvplot.pandas

dev = VDS1022()
dev.set_sampling_rate(40e3)
dev.set_channel(CH1, DC, range='10v', offset=-0.4, probe='x10')
dev.set_channel(CH2, DC, range='10v', offset=-0.4, probe='x10')

df = dev.read('2s').to_dataframe()
df.hvplot(rasterize=True).opts(width=700, ylim=dev.ylim(), colorbar=False)

Read continuous samples on a trigger

In [ ]:
from vds1022 import *
import hvplot.pandas

dev = VDS1022()
dev.set_sampling_rate(40e3)
dev.set_channel(CH1, DC, range='10v', offset=0, probe='x10')
dev.set_channel(CH2, DC, range='10v', offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='2v', alternate=False, sweep=ONCE)

df = dev.read('500ms', pre='50ms').to_dataframe()
df.hvplot(rasterize=True).opts(width=800, ylim=dev.ylim(), colorbar=False)